In [1]:
#!/usr/bin/env python3

import sys
from scapy.all import *
from time import *

import statistics
from statistics import mode, mean


In [48]:

def isTimeExceeded(pkt):
    return pkt[ICMP].type == 11

def rttsXX(url, repCount, ttlRange):
    responses = {}
    for i in range(repCount):
        for ttl in ttlRange:
            probe = IP(dst=url, ttl=ttl) / ICMP()
            t_i = time()
            ans = sr1(probe, verbose=False, timeout=0.8)
            t_f = time()
            rtt = (t_f - t_i)*1000
            if (ans is not None) and isTimeExceeded(ans):

                if ttl not in responses: 
                    responses[ttl] = []
                responses[ttl].append((ans.src, rtt))
        #print(responses)
        #print()    
    return responses

def mostFrequentIPSAndTheirMeanRTTs(rtts):
    mostFrequentIPs = { key : mode([ p[0] for p in value ]) for key, value in rtts.items() }
    
    filteredResponsesByMostFrequent = { key : [x for x in value if x[0]==mostFrequentIPs[key]] for key, value in rtts.items() }
    
    meanRTTs = { key : mean([ p[1] for p in value ]) for key, value in filteredResponsesByMostFrequent.items() }
    
    return mostFrequentIPs, meanRTTs

def getStepsXX(RTTsByTTL, mostFrequentIPs):
    totalRTTs = RTTsByTTL.copy()
    RTTsOfSteps = {}
    IPsOfSteps = mostFrequentIPs.copy()
    sortedKeys = sorted(RTTsByTTL.keys())
    for idx, TTL in enumerate(sortedKeys):
        if idx == 0:
            RTTsOfSteps[TTL] = RTTsByTTL[TTL]
        else:
            for backwardsSkip in range(1,idx+1):
                if (idx - backwardsSkip == 0):
                    diff = RTTsByTTL[TTL] 
                else:
                    previousTTL = sortedKeys[idx-backwardsSkip]
                    if previousTTL not in RTTsOfSteps:
                        continue
                    diff = RTTsByTTL[TTL] - RTTsByTTL[previousTTL]
                if diff > 0:
                    break
                RTTsOfSteps.pop(previousTTL)
                IPsOfSteps.pop(previousTTL)
                totalRTTs.pop(previousTTL)
            RTTsOfSteps[TTL] = diff
    return RTTsOfSteps, IPsOfSteps, totalRTTs

def traceroute(url):
    responses = rttsXX(url, 3, range(1,30))
    
    mostFrequentIPs , meanRTTs = mostFrequentIPSAndTheirMeanRTTs(responses)
    
    steps , IPsOfSteps , totalRTTsOfIPs= getStepsXX(meanRTTs, mostFrequentIPs)
    
    return steps, IPsOfSteps , totalRTTsOfIPs

- Universidades:
    - Heidelberg Universität
        www.uni-heidelberg.de
    - 北京大学
        www.pku.edu.cn
    - University of Cape Town
        www.uct.ac.za

In [49]:
steps, IPsOfSteps, totalRTTsOfIPs = traceroute("www.pku.edu.cn")
print()
print(steps)
print(IPsOfSteps)


{1: 56.33624394734701, 7: 70.01018524169922, 9: 88.50932121276855, 10: 22.495587666829437, 11: 122.50951925913492, 12: 115.60320854187012, 24: 232.22243785858154}
{1: '192.168.0.1', 7: '195.22.220.56', 9: '149.3.181.65', 10: '129.250.4.202', 24: '202.112.41.2', 11: '129.250.2.12', 12: '129.250.6.177'}
